In [ ]:
import sys
import os
from src.botEnv import botEnv
import filepath as file
import random
import neat
import pybullet as p
import time
import math
import numpy as np



In [ ]:
###Alle möglichen Einstellungen zu NEAT außer der Config File können hier gemacht werden


goalPositions = 'Pos3-Axis'        #Name des Files mit den Zielpositionen
conf = 'config.txt'                #Name der Config file die verwendet werden soll
globalGoal = []                    #globale Variable für die Zielpositionen
fitnessDevelopment = []            #globale Variable zum Verlauf der Fitness
bestGenome = []                    #speicher das beste genome ab
numAxis = 3                       #Anzahl der Achsen, die der Roboter bewegen kann. 1,2,3,6,7
maxDist = 1.842                    #maixmale Distanz zweier Punkte, hier über Roboter Bewegungs Test bestimmbar
FF = 2                            #welche Fitnessfunktion verwendet werden soll, 1 oder 2
orVer = False                      #ob die Orientierung mit betrachtet werden soll
maxStep = 400                      #Simulationsschritte Pro Punkt
GUI = True                         #GUI anzeigen oder nicht. Beim Pixelflux unbedignt auf False setzen
numOfPoints = 10                   #Punkte die pro Generation angefahren werden sollen
maxGenerations = 1000              #Anzahln der Generationen, nach der spätestens abgebrochen werden soll

In [ ]:
#Hilfsfunktionen
def setGlobalGoal(file):
    global globalGoal
    with open(file, "rb") as f: #welche Zielpositionen geladen werden sollen
        globalGoal = pickle.load(f)
    f.close()

def calcDistance(firstPos, secondPos): # berechnet euklidische Distanz zwischen Punkten
    dist = math.sqrt(((firstPos[0]-secondPos[0])**2)+((firstPos[1]-secondPos[1])**2)+((firstPos[2]-secondPos[2])**2))
    return dist

def getStartPos(num): #bestimmt eine gewisse menge an start Positionen
    pos = []
    rows = num/2
    for i in range(num):
        pos.append([3*(i%10), (i/10)*3, 0])
    return pos

def fitnessFunction1(head, headOrientation, goal, goalOrientation, ver): #in der arbeit beschriebene FF1
        distance = calcDistance(head, goal)
        if ver == 1:
            d_roll = abs(headOrientation[0] - goalOrientation[0])
            d_pitch = abs(headOrientation[1] - goalOrientation[1])
            d_jaw = abs(headOrientation[2] - goalOrientation[2])
            
            d_or = [d_pitch, d_jaw, d_roll]
        else:
            d_or = [0.,0.,0.]
        return distance, d_or

def fitnessFunction2(head, headOrientation, goal, goalOrientation, startDistance, orDistance, orVer): #in der arbeit beschriebenen FF2

    distance = calcDistance(head, goal)
    if startDistance == 0:
        startDistance = 0.001
    score = (1 - (distance/startDistance)) * 100
    if score < 0:
        score = 0
    if orVer:
        d_jaw = abs(headOrientation[0] - goalOrientation[0])
        d_pitch = abs(headOrientation[1] - goalOrientation[1])
        d_roll = abs(headOrientation[2] - goalOrientation[2])
        for i in range(3):
            if orDistance[i] == 0:
                orDistance[i] = 0.01
        score_roll = (1 - d_roll/orDistance[0])
        if score_roll < 0:
            score_roll = 0
        score_pitch = (1 - d_pitch/orDistance[1])
        if score_pitch < 0:
            score_pitch = 0
            score_jaw = (1 - d_jaw/orDistance[2])
        if score_jaw < 0:
            score_jaw = 0


        score_o = ((score_jaw + score_pitch + score_roll) / 3) * 100
        score += score_o
    return score

def cartToSpher(x,y,z): #Umrechnung in das für den Roboter angepasste Koordinatensystem
    z = z-0.36
    r = math.sqrt(x*x+y*y+z*z)
    a = z/r
    theta = math.acos(a)
    phi = math.atan2(y,x)
    pol = [r,theta,phi]
    return pol

def getGoalPosition(num, orVer): #Bestimmen der Zielpositionen für eine Generation
    global globalGoal
    goalPos = []
    goalSpher = []
    goalOr = []
    for i in range(num):
        index = int(random.uniform(0, len(globalGoal)))
        pos = globalGoal[index]
        goal = [pos[0],pos[1],pos[2]]
        goalS = cartToSpher(pos[0],pos[1],pos[2])
        if len(pos)>3 and orVer:
            goalO = [pos[3],pos[4],pos[5]]
        else:
            goalO = [0.,0.,0.]
        goalPos.append(goal)
        goalSpher.append(goalS)
        goalOr.append(goalO)
    return goalPos, goalSpher, goalOr

In [ ]:
#Funktion zur Evaluation der Genome
def evalGenome(genomes, config):


    
    
    global bestGenome
    global fitnessVerlauf
    ge = []
    nets = []
    bots = []
    distances = []

    goalKart, goalSpher, goalOrientation= getGoalPosition(numOfPoints, orVer)
    startPos = getStartPos(len(genomes))

    if GUI:
        p.connect(p.GUI) #Mit GUI
    else:
        p.connect(p.DIRECT) #ohne GUI

    index = 0
    for genome_id, genome in genomes:
        net = neat.nn.FeedForwardNetwork.create(genome,config)
        nets.append(net)
        ge.append(genome)
        start = startPos[index]
        bots.append(botEnv(startPos = start, numAxis = numAxis))
        genome.fitness = 0
        index += 1
    collisions = 0
    index = 0
    stepCount = 0
    step = 0

####Beginn der Simulation### 
    while(step < numOfPoints):
        for bot in bots:
            if not bot.collisionFlag:
                bot.setStepStart()
        while(stepCount<maxStep):
            for i, bot in enumerate(bots):
                if not bot.collisionFlag:
                    endeffektor = bot.getBotHeadPos() #pos kartesische Koordinaten [0] = x, [1] = y, [2] = x
                    endeffektorSpher = bot.getBotSpherical() #pos räumliche Polarkoordinaten [0] = r, [1] = theta, [2] = phi
                    endeffektorOrientation = list(p.getEulerFromQuaternion(p.getLinkState(bot.botId,bot.numJoints-1)[1])) #[0] = roll, [1] = pitch, [2] = yaw
                    #p.getJointState(bot.botId, X)[0] gibt den Zustand der Achse X an, in dem Programm sind Achsen von 0-6 nummeriert
                    #Als zielpunkte stehen goalKart - kartesischen Koordinaten [0] = x, [1] = y, [2] = z
                                        #  goalSpher- räumliche Polarkoordiante [0] = r, [1] = theta, [2] = phi
                                        # goalOrientierung - Zielorientierung des Endeffektor [0] = roll, [1] = pitch, [2] = yaw
###########################################################################################################################################################                          
                    #JE NACH VERSUCH HIER DIE INPUTS ANPASSEN, AN WAS MAN VERWENDEN MÖCHTE, ANZAHL MUSS DER IN DER CONFIG FESTGELEGTEN ENTSPRECHEN
                    output = nets[i].activate((
                                                p.getJointState(bot.botId, 0)[0],
                                                p.getJointState(bot.botId, 1)[0],
                                                p.getJointState(bot.botId, 3)[0],
                                               endeffektorSpher[0]-goalSpher[step][0],
                                               endeffektorSpher[1]-goalSpher[step][1],
                                               endeffektorSpher[2]-goalSpher[step][2]
                                               ))
                    
                    
##############################################################################################################################################################                    
                    if len(p.getContactPoints(bodyA=bot.botId, bodyB=bot.botId)) > 0:   #Kollision Roboter mit sich selbst
                        if not bot.collisionFlag:
                            collisions += 1
                        bot.collisionFlag = True
                    if len(p.getContactPoints(bodyA=bot.botId, bodyB=bot.planeId)) > 0: #Kollision Roboter Bodenplatte
                        if not bot.collisionFlag:
                            collisions += 1                        
                        bot.collisionFlag = True

                    bot.setMovement(output)

                    
                    bot.moveStep()            
            p.stepSimulation()
            time.sleep(1/240)
            stepCount += 1
        stepCount = 0
        print('Step', step)
        index = 0
        for genome_id, genome in genomes:
            if FF == 1:
                if not bots[index].collisionFlag:
                    isPos = bots[index].getBotHeadPos()
                    isOrientation= bots[index].getBotHeadOrientation()
                    score, orientationScore = fitnessFunction1(isPos, isOrientation, goalKart[step], goalOrientation[step], orVer)
                    bots[index].botScore += score
                    bots[index].orientationScore += orientationScore 
                else:
                    bots[index].botScore += maxDist
                    bots[index].orientationScore[0] += 2*math.pi
                    bots[index].orientationScore[1] += math.pi
                    bots[index].orientationScore[2] += 2*math.pi
            elif FF == 2:
                if not bots[index].collisionFlag:
                    dist = calcDistance(bots[index].stepStartPosition, goalKart[step])
                    orDist = bots[index].calcOrientationDistance(bots[index].stepStartOrientation)
                    score = fitnessFunction2(bots[index].getBotHeadPos(), bots[index].getBotHeadOrientation(), goalKart[step], goalOrientation[step], dist, orDist, orVer)
                    bots[index].botScore += score
            index += 1


        step += 1
        index = 0
           
    for i, (genome_id, genome) in enumerate(genomes):
        if FF == 1:
            d_avg = bots[i].botScore/numOfPoints
            d_jaw = bots[i].orientationScore[0] / numOfPoints
            d_pitch = bots[i].orientationScore[1] / numOfPoints
            d_roll = bots[i].orientationScore[2] / numOfPoints
            if orVer == 0:
                genome.fitness = (1 - d_avg/maxDist) * 100
            else:
                genome.fitness = (1 - d_avg/maxDist) * 100 +  (((1-d_jaw/(2*math.pi))+(1-d_roll/(2*math.pi))+(1-d_pitch/(math.pi)))/3) * 100
        
        if FF == 2:
            genome.fitness = bots[i].botScore/numOfPoints
        print('Fitness:', genome.fitness)
    index = 0
    bestFitness = 0
    for genome_id, genome in genomes:
        if genome.fitness > bestFitness:
            bestFitness = genome.fitness
    fitnessDevelopment.append(bestFitness)
    p.disconnect() 


In [ ]:

####Hier wird NEAT gestartet########

global bestGenomes
global fitnessDevelopment
local_dir = file.getPath()
config_path = os.path.join(local_dir, conf)
try:
    import cPickle as pickle  # pylint: disable=import-error
except ImportError:
    import pickle  # pylint: disable=import-error
global pop
config = neat.config.Config(
    neat.DefaultGenome,
    neat.DefaultReproduction,
    neat.DefaultSpeciesSet,
    neat.DefaultStagnation,
    config_path
)
setGlobalGoal(goalPositions) #lädt die Zielpositionen aus der gewählten datei
pop = neat.Population(config)
pop.add_reporter(neat.StdOutReporter(True))
stats = neat.StatisticsReporter()
pop.add_reporter(stats)
p#op.add_reporter(neat.Checkpointer(5)) #Speichert Checkpoints
winner = pop.run(evalGenome, maxGenerations) #hier die maximale Anzahl an Generationen angeben, aktuell 400
    # Save the winner.
with open('winner-ctrnn', 'wb') as f:
    pickle.dump(winner, f)
f.close()
with open('FitnessVerlauf', 'wb') as FV:
    pickle.dump(fitnessDevelopment, FV)
FV.close()
    
print(winner)


